# 需要的包

In [46]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #所有的变量信息都可以打印出来
# pd.options.display.max_columns = None

In [40]:
# pd.options.display.max_rows = None

# 数据处理


## 数据读取

In [47]:
types={'CUPS_CARD_IN': str,'SPEC_DISC_LVL': str,'TRANS_CHNL': str,'SPEC_DISC_TP': str,'CARD_BIN': str,'MCHNT_TP': str}
train = pd.read_csv('./data2/trans_dtl_train1.csv',dtype=types)
test  = pd.read_csv('./data2/trans_dtl_test1.csv',dtype=types)
target = pd.read_csv('./data2/bind_cards1.csv')
target.columns = ['encrypt_card_no']
target['target'] = 1
test['lable']= 0
train['lable']= 1
full_data = [train, test]
all =pd.concat(full_data)
train.info()
train.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312647 entries, 0 to 312646
Data columns (total 33 columns):
SETTLE_DT           312647 non-null int64
encrypt_mchnt_cd    312647 non-null object
TERM_ID             310599 non-null object
MCHNT_TP            311750 non-null object
CONN_IN             312647 non-null int64
LOC_TRANS_TM        312647 non-null int64
STI_TAKEOUT_IN      312647 non-null int64
TRANS_ST            312647 non-null int64
CARD_BIN            312642 non-null object
CARD_PROD           312647 non-null int64
CARD_ATTR           312647 non-null int64
CARD_MEDIA          312647 non-null int64
CARD_BRAND          312647 non-null int64
CARD_RANK           312647 non-null int64
ISS_INS_ID_CD       312647 non-null int64
ACQ_INS_ID_CD       312647 non-null int64
TRANS_ID            312647 non-null object
TRANS_CHNL          312642 non-null object
TRANS_AT            312647 non-null int64
RESP_CD1            312311 non-null object
RESP_CD3            311665 non-null object

,SETTLE_DT,encrypt_mchnt_cd,TERM_ID,MCHNT_TP,CONN_IN,LOC_TRANS_TM,STI_TAKEOUT_IN,TRANS_ST,CARD_BIN,CARD_PROD,...,SPEC_DISC_LVL,encrypt_card_no,CUPS_CARD_IN,TRANS_MD,HCE_PROD_NM,TOUCH_TP,HCE_PROD_IN,NOPWD_PETTY_IN,WHITE_MCHNT_IN,lable
0,20180612,42fc2d6c76d2048c9aded5dbfcd1a9e2,12042365,8220,1,72535,1,1,19620516,0,...,0,7ccca9ac2b0fa257c3f78e4a0a30730c,2,1,0,2,0,0,1,1
1,20180603,f7fcc1f8027ff424327f7f6a2a2da256,01080209,4816,1,175047,1,1,1962156831,3,...,0,82c32cbb47703f4c3e6fd4585bbab5ce,2,2,0,99,0,0,0,1
2,20180511,e552f7d0516b669e04cbecc16b0b6640,45880776,7011,0,73149,1,1,16628268,1,...,0,edd137b31c3112c69c5e9f7d366dd81d,2,2,0,2,0,0,0,1
3,20180602,047e9e99f210f924ba30260eab354531,10000001,8398,1,63544,1,1,1962309101,0,...,0,55342a18c527b31709f9ee9ba2663c59,2,2,Y,99,0,0,1,1
4,20180606,42fc2d6c76d2048c9aded5dbfcd1a9e2,12042426,8220,1,124212,1,1,19620516,0,...,0,953664ea99b5b2b3fddb08732a358aa7,2,1,0,2,0,0,1,1


## 数据统计

In [80]:
all.shape
train.shape
test.shape
target['encrypt_card_no'].nunique()
test['encrypt_card_no'].nunique()
train['encrypt_card_no'].nunique()
all['encrypt_card_no'].nunique()

(1338952, 32)

(312647, 42)

(1026305, 42)

1199

35001

11200

46201

## 提取简易特征 


### 数字特征

In [3]:
train['SETTLE_DT1'] =pd.to_datetime(train['SETTLE_DT'], format='%Y%m%d')
train['date_diff'] = (pd.Timestamp('2018-08-03')- train['SETTLE_DT1']).dt.days
# train['TRANS_AT1'] = np.log1p(train['TRANS_AT'] )
train['LOC_TRANS_TM1']  = train['LOC_TRANS_TM'] .apply(lambda s: str(s)[:-4])

In [50]:
all['SETTLE_DT1'] =pd.to_datetime(all['SETTLE_DT'], format='%Y%m%d')
all['date_diff'] = (pd.Timestamp('2018-08-03')- all['SETTLE_DT1']).dt.days
# test['TRANS_AT1'] = np.log1p(test['TRANS_AT'] )
all['LOC_TRANS_TM1']  = all['LOC_TRANS_TM'] .apply(lambda s: str(s)[:-4])

### 类别特征

#### MCC之下，商户类型又分为了3个层次的不同类别

In [51]:
mcc = pd.read_csv('./data2/Para/PARA_DIM_MCC.txt',encoding='gb2312',header=0)
all_to=pd.merge(all,mcc[['MCHNT_TP','MCC_DESC','MCC_GRP_DESC1','MCC_GRP_DESC2','MCC_GRP_DESC3']]  , on='MCHNT_TP', how='left')


In [58]:
mcc['MCC_GRP_DESC1'].nunique()
mcc['MCC_GRP_DESC2'].nunique()
mcc['MCC_GRP_DESC3'].nunique()
train['MCHNT_TP'].nunique()

30

15

4

282

#### CARD_BRAND 后续将visa，jcb等归为一类。

In [59]:
CARD_BRAND = pd.read_csv('./data2/Para/PARA_CARD_BRAND.txt',encoding='gb2312',header=0)
CARD_BRAND


,CARD_BRAND,CARD_BRAND_NM,REC_UPD_TS
0,0,其它,2008-11-14-13.31.18.000000
1,1,6字头银标,2008-11-14-13.31.18.000000
2,2,6字头非标,2008-11-14-13.31.18.000000
3,3,银联9字头,2008-11-14-13.31.18.000000
4,4,VISA卡,2008-11-14-13.31.18.000000
5,5,MASTER卡,2008-11-14-13.31.18.000000
6,6,JCB,2008-11-14-13.31.18.000000
7,7,美运卡,2008-11-14-13.31.18.000000
8,8,其它卡BIN,2008-11-14-13.31.18.000000


In [52]:
DIM_TRANS = pd.read_csv('./data2/Para/PARA_DIM_TRANS.txt',encoding='gb2312',header=0)
# DIM_TRANS
# train=pd.merge(train,mcc[['MCHNT_TP','MCC_DESC','MCC_GRP_DESC1','MCC_GRP_DESC2','MCC_GRP_DESC3']]  , on='MCHNT_TP', how='left')
DIM_TRANS['TRANS_DESC'].nunique()
DIM_TRANS['TRANS_GRP_DESC1'].nunique()
DIM_TRANS['TRANS_GRP_DESC2'].nunique()
DIM_TRANS['TRANS_GRP_DESC3'].nunique()
all['TRANS_ID'].nunique()
all_to=pd.merge(all_to,DIM_TRANS[['TRANS_ID','TRANS_GRP_DESC1','TRANS_GRP_DESC2','TRANS_GRP_DESC3']]  , on='TRANS_ID', how='left')


455

79

21

6

82

In [61]:
DIM_RESP = pd.read_csv('./data2/Para/PARA_DIM_RESP.txt',encoding='gb2312',header=0)
DIM_RESP

,resp_cd,RESP_DESC,rec_crt_ts,rec_upd_ts
0,42,无此账户,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
1,62,受限制的卡,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
2,25,找不到原始交易,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
3,64,原始金额不正确,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
4,97,ATM/POS终端号找不到,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
5,22,怀疑操作有误,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
6,68,收到的回答太迟,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
7,33,过期的卡（没收卡）,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
8,16,批准更新第三磁道,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
9,A8,PIN密钥错,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000


In [94]:
all_to.shape

(1338952, 42)

In [62]:
train['ISS_INS_ID_CD'].nunique()
train['RESP_CD4'].nunique()

468

44

### 根据卡号进行聚合

In [63]:
train.columns

Index(['SETTLE_DT', 'encrypt_mchnt_cd', 'TERM_ID', 'MCHNT_TP', 'CONN_IN',
       'LOC_TRANS_TM', 'STI_TAKEOUT_IN', 'TRANS_ST', 'CARD_BIN', 'CARD_PROD',
       'CARD_ATTR', 'CARD_MEDIA', 'CARD_BRAND', 'CARD_RANK', 'ISS_INS_ID_CD',
       'ACQ_INS_ID_CD', 'TRANS_ID', 'TRANS_CHNL', 'TRANS_AT', 'RESP_CD1',
       'RESP_CD3', 'RESP_CD4', 'SPEC_DISC_TP', 'SPEC_DISC_LVL',
       'encrypt_card_no', 'CUPS_CARD_IN', 'TRANS_MD', 'HCE_PROD_NM',
       'TOUCH_TP', 'HCE_PROD_IN', 'NOPWD_PETTY_IN', 'WHITE_MCHNT_IN',
       'SETTLE_DT1', 'date_diff', 'TRANS_AT1', 'LOC_TRANS_TM1', 'MCC_DESC',
       'MCC_GRP_DESC1', 'MCC_GRP_DESC2', 'MCC_GRP_DESC3', 'TRANS_GRP_DESC1',
       'TRANS_GRP_DESC2', 'TRANS_GRP_DESC3'],
      dtype='object')

未处理的字段 encrypt_mchnt_cd TERM_ID

In [85]:
all.columns

Index(['SETTLE_DT', 'encrypt_mchnt_cd', 'TERM_ID', 'MCHNT_TP', 'CONN_IN',
       'LOC_TRANS_TM', 'STI_TAKEOUT_IN', 'TRANS_ST', 'CARD_BIN', 'CARD_PROD',
       'CARD_ATTR', 'CARD_MEDIA', 'CARD_BRAND', 'CARD_RANK', 'ISS_INS_ID_CD',
       'ACQ_INS_ID_CD', 'TRANS_ID', 'TRANS_CHNL', 'TRANS_AT', 'RESP_CD1',
       'RESP_CD3', 'RESP_CD4', 'SPEC_DISC_TP', 'SPEC_DISC_LVL',
       'encrypt_card_no', 'CUPS_CARD_IN', 'TRANS_MD', 'HCE_PROD_NM',
       'TOUCH_TP', 'HCE_PROD_IN', 'NOPWD_PETTY_IN', 'WHITE_MCHNT_IN',
       'SETTLE_DT1', 'date_diff', 'LOC_TRANS_TM1', 'TRANS_GRP_DESC1',
       'TRANS_GRP_DESC2', 'TRANS_GRP_DESC3'],
      dtype='object')

In [53]:
types={'CUPS_CARD_IN': str,'SPEC_DISC_LVL': str,'TRANS_CHNL': str,'SPEC_DISC_TP': str,'CARD_BIN': str,'MCHNT_TP': str}

all_together =(all_to.groupby(['encrypt_card_no'])
                  .agg({
                      'lable': [  'min'],
                      'date_diff': [ 'mean', 'min', 'max'],
                      'TRANS_AT': ['sum', 'mean', 'min', 'max','count'],
                      'HCE_PROD_NM': ['min', 'max', 'nunique'],
                       'MCC_DESC': [  'last','first', 'nunique'],
                      'MCC_GRP_DESC3': [  'last','first', 'nunique'],
                      'MCC_GRP_DESC1': [  'last','first', 'nunique'],
                      'MCC_GRP_DESC2': [  'last','first', 'nunique'],
                      'CONN_IN': ['min', 'nunique'],
                      'LOC_TRANS_TM1': ['min', 'max','nunique','last'],
                      'STI_TAKEOUT_IN': ['max','nunique'],
                      'TRANS_ST': ['min', 'max','last','nunique'],
                      'CARD_PROD': ['last','nunique'],
                      'CARD_ATTR': ['last','nunique'],
                      'CARD_MEDIA': ['last','nunique'],
                      'CARD_BRAND': ['last','nunique'],
                      'CARD_RANK': ['nunique', 'max'],
                      'TRANS_CHNL': [  'last','first', 'nunique'],
                      'TRANS_GRP_DESC1': [  'last','first', 'nunique'],
                      'TRANS_GRP_DESC2': [ 'last','first', 'nunique'],
                      'TRANS_GRP_DESC3': [  'last','first', 'nunique'],
                      'RESP_CD3': [  'last','first', 'nunique'],
                      'SPEC_DISC_TP': [ 'last','first', 'nunique'],
                      'SPEC_DISC_LVL': [  'last','first', 'nunique'],
                      'CUPS_CARD_IN': [  'last', 'nunique'],
                      'TRANS_MD': [  'max','min','last','nunique'],
                      'TOUCH_TP': [ 'first','last', 'nunique'],
                      'HCE_PROD_IN': [  'last','nunique'],
                      'NOPWD_PETTY_IN': [  'last','nunique'],
                      'WHITE_MCHNT_IN': [  'last','nunique']
                  }))

In [54]:
all_together.fillna('MISSING',inplace = True)
all_together =all_together.fillna(value=0)

### rename column

In [55]:
all_together.columns = ["_".join(x) for x in all_together.columns.ravel()]
all_together=all_together.reset_index(level=['encrypt_card_no'],drop=False)
all_together.head(2)

,encrypt_card_no,lable_min,date_diff_mean,date_diff_min,date_diff_max,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,...,TRANS_MD_nunique,TOUCH_TP_first,TOUCH_TP_last,TOUCH_TP_nunique,HCE_PROD_IN_last,HCE_PROD_IN_nunique,NOPWD_PETTY_IN_last,NOPWD_PETTY_IN_nunique,WHITE_MCHNT_IN_last,WHITE_MCHNT_IN_nunique
0,0003c2a9c1a640ca966f4f696d5ae5d4,0,49.000,2,92,639071,15215.976,0,300000,42,...,1,1,99,2,0,1,0,1,0,1
1,0003e33867a72bd47dca072e2958e613,0,14.636,0,35,589900,53627.273,0,139900,11,...,2,1,2,2,0,1,0,1,0,1


## 打标签


###  直接打标签

根据bind_cards1.csv打标签，绑定云闪付的为1，不绑定的为0

In [63]:
all_together.columns

Index(['encrypt_card_no', 'lable_min', 'date_diff_mean', 'date_diff_min',
       'date_diff_max', 'TRANS_AT_sum', 'TRANS_AT_mean', 'TRANS_AT_min',
       'TRANS_AT_max', 'TRANS_AT_count', 'HCE_PROD_NM_min', 'HCE_PROD_NM_max',
       'HCE_PROD_NM_nunique', 'MCC_DESC_last', 'MCC_DESC_first',
       'MCC_DESC_nunique', 'MCC_GRP_DESC3_last', 'MCC_GRP_DESC3_first',
       'MCC_GRP_DESC3_nunique', 'MCC_GRP_DESC1_last', 'MCC_GRP_DESC1_first',
       'MCC_GRP_DESC1_nunique', 'MCC_GRP_DESC2_last', 'MCC_GRP_DESC2_first',
       'MCC_GRP_DESC2_nunique', 'CONN_IN_min', 'CONN_IN_nunique',
       'LOC_TRANS_TM1_min', 'LOC_TRANS_TM1_max', 'LOC_TRANS_TM1_nunique',
       'LOC_TRANS_TM1_last', 'STI_TAKEOUT_IN_max', 'STI_TAKEOUT_IN_nunique',
       'TRANS_ST_min', 'TRANS_ST_max', 'TRANS_ST_last', 'TRANS_ST_nunique',
       'CARD_PROD_last', 'CARD_PROD_nunique', 'CARD_ATTR_last',
       'CARD_ATTR_nunique', 'CARD_MEDIA_last', 'CARD_MEDIA_nunique',
       'CARD_BRAND_last', 'CARD_BRAND_nunique', 'CARD_RAN

In [100]:
x_together =all_together.drop(columns=['encrypt_card_no','lable_min'])

In [101]:
x_together.columns

Index(['date_diff_mean', 'date_diff_min', 'date_diff_max', 'TRANS_AT_sum',
       'TRANS_AT_mean', 'TRANS_AT_min', 'TRANS_AT_max', 'TRANS_AT_count',
       'HCE_PROD_NM_min', 'HCE_PROD_NM_max', 'HCE_PROD_NM_nunique',
       'MCC_DESC_last', 'MCC_DESC_first', 'MCC_DESC_nunique',
       'MCC_GRP_DESC3_last', 'MCC_GRP_DESC3_first', 'MCC_GRP_DESC3_nunique',
       'MCC_GRP_DESC1_last', 'MCC_GRP_DESC1_first', 'MCC_GRP_DESC1_nunique',
       'MCC_GRP_DESC2_last', 'MCC_GRP_DESC2_first', 'MCC_GRP_DESC2_nunique',
       'CONN_IN_min', 'CONN_IN_nunique', 'LOC_TRANS_TM1_min',
       'LOC_TRANS_TM1_max', 'LOC_TRANS_TM1_nunique', 'LOC_TRANS_TM1_last',
       'STI_TAKEOUT_IN_max', 'STI_TAKEOUT_IN_nunique', 'TRANS_ST_min',
       'TRANS_ST_max', 'TRANS_ST_last', 'TRANS_ST_nunique', 'CARD_PROD_last',
       'CARD_PROD_nunique', 'CARD_ATTR_last', 'CARD_ATTR_nunique',
       'CARD_MEDIA_last', 'CARD_MEDIA_nunique', 'CARD_BRAND_last',
       'CARD_BRAND_nunique', 'CARD_RANK_nunique', 'CARD_RANK_max',
   

In [102]:
x_together_dummy = pd.get_dummies(x_together)

In [99]:
x_together_dummy.shape

(46201, 47166)

In [103]:
x_together_dummy['lable_min']=all_together[ 'lable_min']

In [104]:
x_together_dummy['encrypt_card_no']=all_together[ 'encrypt_card_no'] 

In [105]:
test_g=x_together_dummy[x_together_dummy['lable_min']==0]

In [106]:
train_g = x_together_dummy[x_together_dummy['lable_min']==1]

In [96]:
train_g.shape

(11200, 47166)

In [107]:
train_v=pd.merge(train_g, target, on='encrypt_card_no', how='outer')
train_v =train_v.fillna(value=0)
train_v.shape

(11200, 966)

In [14]:
train_v.columns

Index(['encrypt_card_no', 'date_diff_mean', 'date_diff_min', 'date_diff_max',
       'TRANS_AT_sum', 'TRANS_AT_mean', 'TRANS_AT_min', 'TRANS_AT_max',
       'TRANS_AT_count', 'HCE_PROD_NM_min', 'HCE_PROD_NM_max',
       'HCE_PROD_NM_nunique', 'MCC_DESC_last', 'MCC_DESC_first',
       'MCC_DESC_nunique', 'MCC_GRP_DESC3_last', 'MCC_GRP_DESC3_first',
       'MCC_GRP_DESC3_nunique', 'MCC_GRP_DESC1_last', 'MCC_GRP_DESC1_first',
       'MCC_GRP_DESC1_nunique', 'MCC_GRP_DESC2_last', 'MCC_GRP_DESC2_first',
       'MCC_GRP_DESC2_nunique', 'CONN_IN_min', 'CONN_IN_nunique',
       'LOC_TRANS_TM1_min', 'LOC_TRANS_TM1_max', 'LOC_TRANS_TM1_nunique',
       'LOC_TRANS_TM1_last', 'STI_TAKEOUT_IN_max', 'STI_TAKEOUT_IN_nunique',
       'TRANS_ST_min', 'TRANS_ST_max', 'TRANS_ST_last', 'TRANS_ST_nunique',
       'CARD_PROD_last', 'CARD_PROD_nunique', 'CARD_ATTR_last',
       'CARD_ATTR_nunique', 'CARD_MEDIA_last', 'CARD_MEDIA_nunique',
       'CARD_BRAND_last', 'CARD_BRAND_nunique', 'CARD_RANK_nunique',
 

In [78]:
test_g.shape

(35001, 83)

In [108]:
y = train_v[['target']]
# y['target'] = y['target'].astype(str).astype(int)

In [109]:
train_v = train_v.drop(columns=['encrypt_card_no','lable_min', 'target'])

In [110]:
test_v = test_g.drop(columns=['lable_min'])

In [47]:
all_together.columns

Index(['encrypt_card_no', 'date_diff_mean', 'date_diff_min', 'date_diff_max',
       'TRANS_AT_sum', 'TRANS_AT_mean', 'TRANS_AT_min', 'TRANS_AT_max',
       'TRANS_AT_count', 'HCE_PROD_NM_min', 'HCE_PROD_NM_max',
       'HCE_PROD_NM_nunique', 'MCC_DESC_last', 'MCC_DESC_first',
       'MCC_DESC_nunique', 'MCC_GRP_DESC3_last', 'MCC_GRP_DESC3_first',
       'MCC_GRP_DESC3_nunique', 'MCC_GRP_DESC1_last', 'MCC_GRP_DESC1_first',
       'MCC_GRP_DESC1_nunique', 'MCC_GRP_DESC2_last', 'MCC_GRP_DESC2_first',
       'MCC_GRP_DESC2_nunique', 'CONN_IN_min', 'CONN_IN_nunique',
       'LOC_TRANS_TM1_min', 'LOC_TRANS_TM1_max', 'LOC_TRANS_TM1_nunique',
       'LOC_TRANS_TM1_last', 'STI_TAKEOUT_IN_max', 'STI_TAKEOUT_IN_nunique',
       'TRANS_ST_min', 'TRANS_ST_max', 'TRANS_ST_last', 'TRANS_ST_nunique',
       'CARD_PROD_last', 'CARD_PROD_nunique', 'CARD_ATTR_last',
       'CARD_ATTR_nunique', 'CARD_MEDIA_last', 'CARD_MEDIA_nunique',
       'CARD_BRAND_last', 'CARD_BRAND_nunique', 'CARD_RANK_nunique',
 

In [111]:
test_v.set_index('encrypt_card_no', inplace=True)

In [36]:
all_together_test.head(3)

,date_diff_mean,date_diff_min,date_diff_max,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,HCE_PROD_NM_min,HCE_PROD_NM_max,...,TRANS_MD_nunique,TOUCH_TP_first,TOUCH_TP_last,TOUCH_TP_nunique,HCE_PROD_IN_last,HCE_PROD_IN_nunique,NOPWD_PETTY_IN_last,NOPWD_PETTY_IN_nunique,WHITE_MCHNT_IN_last,WHITE_MCHNT_IN_nunique
encrypt_card_no,,,,,,,,,,,,,,,,,,,,,
0003c2a9c1a640ca966f4f696d5ae5d4,49.000,2,92,639071,15215.976,0,300000,42,0,0,...,1,1,99,2,0,1,0,1,0,1
0003e33867a72bd47dca072e2958e613,14.636,0,35,589900,53627.273,0,139900,11,0,0,...,2,1,2,2,0,1,0,1,0,1
000bfc5bd843adc837a4c1a266bd4dae,34.769,2,71,910000,70000.000,0,330000,13,0,0,...,1,1,1,1,0,1,0,1,0,1


In [112]:
test_v.shape

(35001, 963)

In [53]:
all_together.shape

(11200, 82)

In [89]:
test_v.shape

(35001, 81)

In [114]:
train_v.shape

(11200, 963)

In [39]:
test_dummy = pd.get_dummies(test_v)

In [76]:
test_dummy.shape

(35001, 938)

0-无产品          1-Apple Pay       2-HCE       3-三星pay mst  4-三星pay ic  5-线上大商户          6-华为 

7-小米          8-中兴          Y-主扫         Z-被扫

# 结果文件

In [26]:
def print_output(y,y_pred):
    precision = metrics.precision_score(y, y_pred)
    recall = metrics.recall_score(y, y_pred)
    F1 = 2*precision*recall/(precision + recall)

    print("Precision : %.4g" % precision)
    print("Recall: %f" % recall)
    print("F1: %f" % F1)
    return precision,recall,F1

In [27]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [28]:
def standard_data(data):
    scaler = preprocessing.StandardScaler()
    scaler.fit(data)
    data_normorlize = scaler.transform(data)

    return data_normorlize

In [29]:
def train_classifier(X, Y):
    gbm0 = GradientBoostingClassifier(random_state=10)
    gbm0.fit(X,Y)
    return gbm0

In [30]:
def main(n,Test_size,X_total,Y_total,ifnorm):
    result_ar = np.ones((n,3))
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=Test_size,shuffle=True, random_state=i)
        if ifnorm:
            X_train = standard_data(X_train)
            X_test = standard_data(X_test)
        GBDT1 = train_classifier(X_train, y_train)
        y_pred = GBDT1.predict(X_test)
        result_ar[i,:] = print_output(y_test, y_pred)
    return  result_ar

In [125]:
train_v.to_csv('./train_v.csv',header=False,index=False)
test_v.to_csv('./test_v.csv',header=False,index=False)
y['target'].to_csv('./target.csv',header=False,index=False)

In [115]:
result = main(5, 0.3, train_v, y['target'], 0)

Precision : 0.9866
Recall: 0.821229
F1: 0.896341
Precision : 0.9808
Recall: 0.871795
F1: 0.923077
Precision : 0.981
Recall: 0.846995
F1: 0.909091
Precision : 0.9935
Recall: 0.819892
F1: 0.898380
Precision : 0.9708
Recall: 0.856734
F1: 0.910198


In [116]:

GBDT2 = train_classifier(train_v, y['target'])

In [119]:
train_v.shape
test_v.shape

(11200, 963)

(35001, 963)

In [120]:
X_test_feature_data = np.array(test_v)
Y_test_pred = GBDT2.predict(X_test_feature_data)

In [123]:
result_test = pd.DataFrame(test_v.index)
result_test["pred"] = Y_test_pred
card_no = np.array(result_test.iloc[Y_test_pred==1]['encrypt_card_no'])
card_no = pd.DataFrame(card_no)
card_no.to_csv('./card_nocard_no927.csv',header=False,index=False)

In [124]:
card_no.shape

(3971, 1)

In [87]:
train_dummy.columns

Index(['date_diff_mean', 'date_diff_min', 'date_diff_max', 'TRANS_AT1_max',
       'TRANS_AT_sum', 'TRANS_AT_mean', 'TRANS_AT_min', 'TRANS_AT_max',
       'TRANS_AT_count', 'HCE_PROD_NM_nunique',
       ...
       'SPEC_DISC_LVL_last_0', 'SPEC_DISC_LVL_last_1',
       'SPEC_DISC_LVL_last_MISSING', 'SPEC_DISC_LVL_first_0',
       'SPEC_DISC_LVL_first_1', 'SPEC_DISC_LVL_first_MISSING',
       'CUPS_CARD_IN_last_0', 'CUPS_CARD_IN_last_2', 'CUPS_CARD_IN_last_3',
       'CUPS_CARD_IN_last_MISSING'],
      dtype='object', length=844)

In [88]:
train_dummy.head(2)

,date_diff_mean,date_diff_min,date_diff_max,TRANS_AT1_max,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,HCE_PROD_NM_nunique,...,SPEC_DISC_LVL_last_0,SPEC_DISC_LVL_last_1,SPEC_DISC_LVL_last_MISSING,SPEC_DISC_LVL_first_0,SPEC_DISC_LVL_first_1,SPEC_DISC_LVL_first_MISSING,CUPS_CARD_IN_last_0,CUPS_CARD_IN_last_2,CUPS_CARD_IN_last_3,CUPS_CARD_IN_last_MISSING
0,54.000,2,81,14.221,4050743,253171.438,2350,1500000,16,1,...,1,0,0,1,0,0,0,1,0,0
1,24.706,2,46,10.820,320780,18869.412,0,50000,17,1,...,1,0,0,1,0,0,0,1,0,0


In [89]:
np.array(train_dummy)

array([[54.        ,  2.        , 81.        , ...,  1.        ,
         0.        ,  0.        ],
       [24.70588235,  2.        , 46.        , ...,  1.        ,
         0.        ,  0.        ],
       [14.2       ,  1.        , 65.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [39.33333333,  2.        , 70.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 2.        ,  2.        ,  2.        , ...,  1.        ,
         0.        ,  0.        ],
       [19.57142857,  2.        , 35.        , ...,  1.        ,
         0.        ,  0.        ]])

In [90]:
y['target'].head(2)

0   0.000
1   0.000
Name: target, dtype: float64

In [92]:
X_train_data = np.array(train_dummy)
Y_train_data = y['target']
import pandas as pd
import numpy as np
import classifiers as cls


###############################only train#################################
lr_norm = 0
svm_norm = 0
glr_norm = 0
dim_para = 0
xgb_norm = 0
train_and_test = 0 # 0 ：只是在训练集上训练模型 1：在测试集上进行测试
if train_and_test == 0:
#     cls.LR_clf(X_train_data,Y_train_data)
#     cls.SVM_clf(X_train_data,Y_train_data)
    cls.gbdt_lr_clf(X_train_data,Y_train_data)
    cls.xgb_clf(X_train_data, Y_train_data)

if train_and_test == 1 :
    lr_card_list = cls.lr_test(X_train_data,Y_train_data,X_test_data,lr_norm,dim_para)
    svm_card_list = cls.SVM_test(X_train_data,Y_train_data,X_test_data,svm_norm,dim_para)
    gblr_card_list = cls.GBDT_LR_test(X_train_data,Y_train_data,X_test_data,glr_norm,dim_para)
    xgb_cad_list = cls.XGBoost_test(X_train_data, Y_train_data, X_test_data, xgb_norm, dim_para)

gbdt_classifier + LR:
n_estimators:10.000000,gblr_p:0.996949,gblr_r:0.825652,gblr_f1:0.902990
gbdt_classifier + LR:
n_estimators:20.000000,gblr_p:0.989343,gblr_r:0.836492,gblr_f1:0.906174
gbdt_classifier + LR:
n_estimators:30.000000,gblr_p:0.990214,gblr_r:0.834822,gblr_f1:0.905598
gbdt_classifier + LR:
n_estimators:40.000000,gblr_p:0.990252,gblr_r:0.834822,gblr_f1:0.905629
gbdt_classifier + LR:
n_estimators:50.000000,gblr_p:0.982597,gblr_r:0.836492,gblr_f1:0.903321


([0.8583333333333333,
  0.8541666666666666,
  0.8291666666666667,
  0.8041666666666667,
  0.7824267782426778,
  0.8708333333333333,
  0.8625,
  0.8375,
  0.8208333333333333,
  0.7907949790794979,
  0.8708333333333333,
  0.8541666666666666,
  0.8333333333333334,
  0.8291666666666667,
  0.7866108786610879,
  0.8666666666666667,
  0.8541666666666666,
  0.8375,
  0.8291666666666667,
  0.7866108786610879,
  0.875,
  0.8583333333333333,
  0.8291666666666667,
  0.8291666666666667,
  0.7907949790794979],
 [1.0,
  1.0,
  0.995,
  0.9897435897435898,
  1.0,
  0.9858490566037735,
  0.9904306220095693,
  0.9804878048780488,
  0.9899497487437185,
  1.0,
  0.9858490566037735,
  0.9903381642512077,
  0.9950248756218906,
  0.9851485148514851,
  0.9947089947089947,
  0.9904761904761905,
  1.0,
  0.9804878048780488,
  0.9802955665024631,
  1.0,
  0.9813084112149533,
  0.976303317535545,
  0.9851485148514851,
  0.9754901960784313,
  0.9947368421052631],
 [0.9237668161434976,
  0.9213483146067416,
  0.904

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:  4.8min finished


每轮迭代运行结果:{'mean_fit_time': array([15.92878957, 18.46737795, 21.42591105, 27.19107323, 31.68348517,
       30.46111598, 34.04000878, 38.17106752]), 'std_fit_time': array([0.53534249, 0.4123166 , 0.26178292, 1.81021551, 1.83573469,
       0.24384182, 1.19342305, 2.16898213]), 'mean_score_time': array([0.12236304, 0.10502901, 0.13459396, 0.16054749, 0.17921557,
       0.21563478, 0.19839678, 0.12797055]), 'std_score_time': array([0.04980898, 0.05688997, 0.03310547, 0.06000308, 0.05919433,
       0.07320937, 0.07176802, 0.07178609]), 'param_n_estimators': masked_array(data=[50, 60, 70, 80, 90, 100, 110, 120],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 50}, {'n_estimators': 60}, {'n_estimators': 70}, {'n_estimators': 80}, {'n_estimators': 90}, {'n_estimators': 100}, {'n_estimators': 110}, {'n_estimators': 120}], 'split0_test_score': array([0.91314031, 0.91555556, 0.91555556, 0.9115

KeyboardInterrupt: 

In [79]:
train_dummy.shape

(11200, 844)

In [80]:
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='pearson', min_periods=1)
    df_not_correlated = ~(df_corr.mask(np.eye(len(df_corr), dtype=bool)).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

uncorrelated_factors = trimm_correlated(train_dummy, 0.8)
uncorrelated_factors

,date_diff_min,TRANS_AT1_max,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,CONN_IN_min,CONN_IN_nunique,LOC_TRANS_TM1_nunique,...,RESP_CD3_first_75,RESP_CD3_first_91,RESP_CD3_first_94,RESP_CD3_first_98,RESP_CD3_first_P1,SPEC_DISC_TP_last_01,SPEC_DISC_TP_last_02,SPEC_DISC_TP_last_04,SPEC_DISC_TP_first_02,SPEC_DISC_TP_first_04
0,2,14.221,4050743,253171.438,2350,1500000,16,0,2,7,...,0,0,0,0,0,0,0,0,0,0
1,2,10.820,320780,18869.412,0,50000,17,0,2,5,...,0,0,0,0,0,0,0,0,0,0
2,1,11.002,304000,30400.000,0,60000,10,0,1,4,...,0,0,0,0,0,0,0,0,0,0
3,2,13.556,2313300,578325.000,0,771100,4,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2,12.345,229900,229900.000,229900,229900,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,2,0.000,0,0.000,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6,1,13.476,4316800,479644.444,235900,712000,9,0,1,6,...,0,0,0,0,0,0,0,0,0,0
7,2,11.930,488020,69717.143,24400,151710,7,0,2,6,...,0,0,0,0,0,0,0,0,0,0
8,2,13.251,1354500,67725.000,5000,568400,20,0,2,4,...,0,0,0,0,0,0,0,0,0,0
9,3,15.520,59342501,1561644.763,-1228,5500000,38,0,2,9,...,0,0,0,0,0,0,0,0,0,0


In [81]:
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=300) # k < n, try something like int(round(n/10.))
# Fit it to your data
selector.fit(train_dummy, y['target']) 

ValueError: Input X must be non-negative.

In [24]:

corr = train_dummy.corr()
# corr.sort_values(["SalePrice"], ascending = False, inplace = True)
corr

,date_diff_mean,date_diff_min,date_diff_max,TRANS_AT1_max,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,HCE_PROD_NM_nunique,...,SPEC_DISC_LVL_last_0,SPEC_DISC_LVL_last_1,SPEC_DISC_LVL_last_MISSING,SPEC_DISC_LVL_first_0,SPEC_DISC_LVL_first_1,SPEC_DISC_LVL_first_MISSING,CUPS_CARD_IN_last_0,CUPS_CARD_IN_last_2,CUPS_CARD_IN_last_3,CUPS_CARD_IN_last_MISSING
date_diff_mean,1.000,0.308,0.858,0.273,0.054,-0.009,-0.049,0.021,0.198,0.029,...,-0.018,0.017,0.008,-0.019,0.018,0.008,0.057,-0.105,0.095,-0.022
date_diff_min,0.308,1.000,0.021,-0.122,-0.023,0.014,0.020,-0.007,-0.087,-0.002,...,0.008,-0.007,-0.001,-0.001,0.002,-0.001,0.140,0.034,-0.069,-0.003
date_diff_max,0.858,0.021,1.000,0.374,0.073,-0.014,-0.065,0.035,0.261,0.049,...,-0.018,0.017,0.006,-0.019,0.018,0.006,0.017,-0.122,0.122,-0.025
TRANS_AT1_max,0.273,-0.122,0.374,1.000,0.128,0.183,0.067,0.161,0.075,0.013,...,0.016,-0.014,-0.012,0.016,-0.014,-0.012,0.020,-0.131,0.131,-0.045
TRANS_AT_sum,0.054,-0.023,0.073,0.128,1.000,0.485,0.030,0.644,0.225,0.000,...,-0.002,0.003,-0.003,-0.001,0.002,-0.003,-0.004,0.009,-0.008,-0.002
TRANS_AT_mean,-0.009,0.014,-0.014,0.183,0.485,1.000,0.734,0.668,-0.020,-0.005,...,0.009,-0.008,-0.004,0.004,-0.003,-0.004,-0.001,0.003,-0.003,-0.002
TRANS_AT_min,-0.049,0.020,-0.065,0.067,0.030,0.734,1.000,0.169,-0.022,-0.011,...,0.002,-0.002,-0.001,-0.001,0.001,-0.001,0.000,0.006,-0.007,-0.001
TRANS_AT_max,0.021,-0.007,0.035,0.161,0.644,0.668,0.169,1.000,0.019,0.012,...,0.007,-0.007,-0.003,0.006,-0.006,-0.003,-0.003,0.005,-0.004,-0.002
TRANS_AT_count,0.198,-0.087,0.261,0.075,0.225,-0.020,-0.022,0.019,1.000,0.069,...,-0.015,0.013,0.010,-0.014,0.013,0.010,-0.018,0.033,-0.030,-0.005
HCE_PROD_NM_nunique,0.029,-0.002,0.049,0.013,0.000,-0.005,-0.011,0.012,0.069,1.000,...,0.003,-0.002,-0.004,0.018,-0.017,-0.004,0.058,0.027,-0.042,-0.002


In [ ]:
a=train.loc[test['HCE_PROD_NM'] != '0'].encrypt_card_no.unique()
s=pd.Series(a)
s.to_csv('2.csv',index=False)

In [137]:
a=test.loc[test['HCE_PROD_NM'] != '0'].encrypt_card_no.unique()
pd.DataFrame(a).to_csv("1.csv",header=False,index=False)